In [61]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from statsmodels.tsa.stattools import adfuller
from abc import ABC, abstractmethod
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

## 1. FEATURE ENGINEERING LAB (The Math)

In [62]:
class FeatureLab:
    """Shared mathematical engine for technical and statistical features."""
    
    @staticmethod
    def get_weights_frac_diff(d, size, threshold=1e-5):
        w = [1.0]
        for k in range(1, size):
            w_k = -w[-1] / k * (d - k + 1)
            w.append(w_k)
        w = np.array(w[::-1])
        w = w[np.abs(w) > threshold]
        return w

    @staticmethod
    def frac_diff_fixed(series, d, window=50):
        # Solves Stationarity Dilemma [cite: 61]
        weights = FeatureLab.get_weights_frac_diff(d, window)
        res = series.rolling(window=len(weights)).apply(lambda x: np.dot(x, weights), raw=True)
        return res

    @staticmethod
    def yang_zhang_volatility(df, window=30):
        # Captures intraday energy/gaps [cite: 82]
        log_ho = (df['High'] / df['Open']).apply(np.log)
        log_lo = (df['Low'] / df['Open']).apply(np.log)
        log_co = (df['Close'] / df['Open']).apply(np.log)
        log_oc = (df['Open'] / df['Close'].shift(1)).apply(np.log)
        log_cc = (df['Close'] / df['Close'].shift(1)).apply(np.log)
        
        rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)
        close_vol = log_cc.rolling(window=window).var()
        open_vol = log_oc.rolling(window=window).var()
        window_rs = rs.rolling(window=window).mean()

        k = 0.34 / (1.34 + (window + 1) / (window - 1))
        return np.sqrt(open_vol + k * window_rs)

    @staticmethod
    def compute_rsi(series, window=14):
        delta = series.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    @staticmethod
    def triple_barrier_labels(prices, vol, pt=1.0, sl=1.0, barrier_window=10):
        """
        Implements the Triple Barrier Method.
        Labels: 1 (Profit Target Hit), -1 (Stop Loss Hit), 0 (Time Limit/Neutral)
        """
        labels = pd.Series(0, index=prices.index)
        # Shift prices to align future outcome with current row
        # However, to avoid look-ahead in features, we usually compute label for row t based on t+1...t+k
        # This function generates the TARGET variable (y) for training.
        
        limit = len(prices) - barrier_window
        p_values = prices.values
        v_values = vol.values
        
        for i in range(limit):
            current_p = p_values[i]
            current_vol = v_values[i]
            
            # Dynamic barriers based on volatility [cite: 215]
            target = current_p * (1 + pt * current_vol)
            stop = current_p * (1 - sl * current_vol)
            
            future_window = p_values[i+1 : i+1+barrier_window]
            
            hit_target = np.where(future_window >= target)[0]
            hit_stop = np.where(future_window <= stop)[0]
            
            first_target = hit_target[0] if len(hit_target) > 0 else barrier_window + 1
            first_stop = hit_stop[0] if len(hit_stop) > 0 else barrier_window + 1
            
            if first_target < first_stop and first_target <= barrier_window:
                labels.iloc[i] = 1
            elif first_stop < first_target and first_stop <= barrier_window:
                labels.iloc[i] = 0 # In Meta-Labeling, we often treat Stop (-1) as 0 (Do Not Trade)
            # Else 0 (Time limit reached or neutral)
            
        return labels

## 2. BASE STRATEGY INFRASTRUCTURE

In [63]:
class BaseStrategy(ABC):
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.data = None
        self.results = None
        self.metrics = {}

    def fetch_data(self, warmup_years=2):
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d")
        warmup_start_dt = start_dt - timedelta(days=warmup_years*365)
        warmup_start_str = warmup_start_dt.strftime("%Y-%m-%d")
        
        try:
            self.data = yf.download(self.ticker, start=warmup_start_str, end=self.end_date, progress=False, auto_adjust=False)
            if isinstance(self.data.columns, pd.MultiIndex): 
                self.data.columns = self.data.columns.get_level_values(0)
            if 'Adj Close' not in self.data.columns: 
                self.data['Adj Close'] = self.data['Close']
            self.data['Returns'] = self.data['Adj Close'].pct_change()
            self.data.dropna(inplace=True)
        except Exception as e:
            print(f"Error fetching {self.ticker}: {e}")
            self.data = pd.DataFrame()

    @abstractmethod
    def generate_signals(self):
        pass

    def run_backtest(self, transaction_cost=0.0005, rebalance_threshold=0.1):
        if self.data is None or self.data.empty: return
        
        backtest_mask = self.data.index >= self.start_date
        df = self.data.loc[backtest_mask].copy()
        if df.empty: return

        # Position Smoothing
        clean_positions = []
        current_pos = 0.0
        raw_signals = df['Signal'].values
        
        for target in raw_signals:
            if abs(target - current_pos) > rebalance_threshold:
                current_pos = target
            clean_positions.append(current_pos)
            
        df['Position'] = clean_positions
        df['Prev_Position'] = df['Position'].shift(1).fillna(0)
        df['Turnover'] = (df['Prev_Position'] - df['Position'].shift(2).fillna(0)).abs()
        df['Gross_Returns'] = df['Prev_Position'] * df['Returns']
        df['Net_Returns'] = df['Gross_Returns'] - (df['Turnover'] * transaction_cost)
        df['Net_Returns'].fillna(0, inplace=True)
        
        df['Cumulative_Strategy'] = (1 + df['Net_Returns']).cumprod()
        df['Cumulative_Market'] = (1 + df['Returns']).cumprod()
        
        roll_max = df['Cumulative_Strategy'].cummax()
        df['Drawdown'] = (df['Cumulative_Strategy'] / roll_max) - 1.0
        
        self.results = df
        
        # Performance Calculation
        total_ret = df['Cumulative_Strategy'].iloc[-1] - 1
        vol = df['Net_Returns'].std() * np.sqrt(252)
        sharpe = (df['Net_Returns'].mean() / df['Net_Returns'].std()) * np.sqrt(252) if vol > 0 else 0
        max_dd = df['Drawdown'].min()
        
        self.metrics = {
            'Total Return': total_ret,
            'Sharpe Ratio': sharpe,
            'Max Drawdown': max_dd
        }
        return df

## 3.  MODELS (V1-V4)

In [64]:
class StrategyV1_Baseline(BaseStrategy):
    """V1: Fixed FracDiff, Standard GMM."""
    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['FracDiff'] = FeatureLab.frac_diff_fixed(df['Adj Close'].apply(np.log), d=0.4, window=50)
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df['Returns_Smoothed'] = df['Returns'].rolling(5).mean()
        df['Vol_Smoothed'] = df['Volatility'].rolling(5).mean()
        df.dropna(inplace=True)
        
        X = df[['Returns_Smoothed', 'Vol_Smoothed']].values
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        gmm = GaussianMixture(n_components=3, random_state=42)
        df['Cluster'] = gmm.fit_predict(X_scaled)
        
        stats = df.groupby('Cluster')['Returns_Smoothed'].mean().sort_values().index
        mapping = {stats[0]: -1, stats[1]: 0, stats[2]: 1}
        df['Regime'] = df['Cluster'].map(mapping)
        
        df['Signal'] = 0
        df.loc[(df['Regime'] == 1) & (df['FracDiff'] > 0), 'Signal'] = 1
        df.loc[(df['Regime'] == 0) & (df['RSI'] < 40), 'Signal'] = 1
        
        target_vol = 0.15 / np.sqrt(252)
        df['Vol_Scaler'] = (target_vol / df['Volatility']).clip(upper=1.5)
        df['Signal'] = df['Signal'] * df['Vol_Scaler']
        self.data = df

In [65]:
class StrategyV2_Advanced(BaseStrategy):
    """V2: Rolling GMM."""
    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['FracDiff'] = FeatureLab.frac_diff_fixed(df['Adj Close'].apply(np.log), d=0.4, window=50)
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df['Returns_Smoothed'] = df['Returns'].rolling(5).mean()
        df['Vol_Smoothed'] = df['Volatility'].rolling(5).mean()
        df.dropna(inplace=True)
        
        df['Regime'] = 0
        window_size, step_size = 504, 126
        preds, indices = [], []
        
        if len(df) > window_size:
            for t in range(window_size, len(df), step_size):
                train = df.iloc[t-window_size:t]
                test = df.iloc[t:t+step_size]
                if test.empty: break
                
                scaler = StandardScaler()
                X_train_s = scaler.fit_transform(train[['Returns_Smoothed', 'Vol_Smoothed']].values)
                X_test_s = scaler.transform(test[['Returns_Smoothed', 'Vol_Smoothed']].values)
                
                gmm = GaussianMixture(n_components=3, random_state=42).fit(X_train_s)
                train['Clust'] = gmm.predict(X_train_s)
                stats = train.groupby('Clust')['Returns_Smoothed'].mean().sort_values().index
                mapping = {stats[0]: -1, stats[1]: 0, stats[2]: 1}
                
                preds.extend([mapping[x] for x in gmm.predict(X_test_s)])
                indices.extend(test.index)
            
            df.loc[indices, 'Regime'] = pd.Series(preds, index=indices)
        
        df['Signal'] = 0
        df.loc[(df['Regime'] == 1) & (df['FracDiff'] > 0), 'Signal'] = 1
        df.loc[(df['Regime'] == 0) & (df['RSI'] < 45), 'Signal'] = 1
        
        target_vol = 0.15 / np.sqrt(252)
        df['Vol_Scaler'] = (target_vol / df['Volatility']).clip(upper=1.0)
        df['Signal'] = df['Signal'] * df['Vol_Scaler']
        self.data = df

In [66]:
class StrategyV3_Macro(BaseStrategy):
    """V3: Macro (SPY) Filter."""
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.spy_data = None

    def fetch_data(self, warmup_years=2):
        super().fetch_data(warmup_years)
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        try:
            spy = yf.download("SPY", start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=False)
            if isinstance(spy.columns, pd.MultiIndex): spy.columns = spy.columns.get_level_values(0)
            self.spy_data = spy[['Adj Close']].rename(columns={'Adj Close': 'SPY_Price'})
        except: pass

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        if self.spy_data is not None:
            df = df.join(self.spy_data, how='left')
            df['SPY_MA200'] = df['SPY_Price'].rolling(window=200).mean()
            df['Macro_Bull'] = df['SPY_Price'] > df['SPY_MA200']
        else:
            df['Macro_Bull'] = True
            
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['FracDiff'] = FeatureLab.frac_diff_fixed(df['Adj Close'].apply(np.log), d=0.4, window=50)
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df.dropna(inplace=True)
        
        df['Signal'] = 0
        df.loc[(df['FracDiff'] > 0), 'Signal'] = 1
        df.loc[df['Macro_Bull'] == False, 'Signal'] = 0
        
        target_vol = 0.15 / np.sqrt(252)
        df['Signal'] = df['Signal'] * (target_vol / df['Volatility']).clip(upper=1.5)
        self.data = df

In [67]:
class StrategyV4_Meta(BaseStrategy):
    """V4: Dynamic Profiling with OBV."""
    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['FracDiff'] = FeatureLab.frac_diff_fixed(df['Adj Close'].apply(np.log), d=0.4, window=50)
        df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
        df['OBV_Trend'] = df['OBV'].rolling(50).mean()
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df.dropna(inplace=True)
        
        df['Signal'] = 0
        # Trend
        df.loc[(df['FracDiff'] > 0) & (df['OBV'] > df['OBV_Trend']), 'Signal'] = 1
        # Reversion
        df.loc[(df['RSI'] < 30), 'Signal'] = 1
        
        target_vol = 0.15 / np.sqrt(252)
        df['Signal'] = df['Signal'] * (target_vol / df['Volatility']).clip(upper=1.5)
        self.data = df

In [68]:
class StrategyV5_KalmanState(BaseStrategy):
    """
    V5 (Formerly V10): Kalman Filter + Macro Filter + Volatility Burst Control.
    Uses Kalman Filter for noise-free slope estimation[cite: 151].
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.spy_data = None

    def fetch_data(self, warmup_years=2):
        super().fetch_data(warmup_years)
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        try:
            spy = yf.download("SPY", start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=False)
            if isinstance(spy.columns, pd.MultiIndex): spy.columns = spy.columns.get_level_values(0)
            spy['Macro_Trend'] = (spy['Adj Close'] > spy['Adj Close'].rolling(200).mean()).astype(int)
            self.spy_data = spy[['Macro_Trend']]
        except: pass

    def _apply_kalman_filter(self, prices):
        x = prices.values
        n = len(x)
        state = np.zeros(n)
        slope = np.zeros(n)
        state[0] = x[0]
        P, Q, R = 1.0, 0.001, 0.1
        
        for t in range(1, n):
            pred_state = state[t-1] + slope[t-1]
            pred_P = P + Q
            measurement = x[t]
            residual = measurement - pred_state
            K = pred_P / (pred_P + R)
            state[t] = pred_state + K * residual
            slope[t] = 0.9 * slope[t-1] + 0.1 * (state[t] - state[t-1])
            P = (1 - K) * pred_P
        return pd.Series(slope, index=prices.index)

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        if self.spy_data is not None:
            df = df.join(self.spy_data, how='left').fillna(method='ffill')
        else:
            df['Macro_Trend'] = 1 
            
        log_prices = np.log(df['Adj Close'])
        df['Kalman_Slope'] = self._apply_kalman_filter(log_prices)
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['Vol_Change'] = df['Volatility'].diff()
        
        df.dropna(inplace=True)
        
        # Primary Logic
        df['Signal'] = 0.0
        long_condition = (df['Kalman_Slope'] > 0) & (df['Macro_Trend'] == 1)
        df.loc[long_condition, 'Signal'] = 1
        
        # Vol Targeting & Burst Protection
        target_vol = 0.15 / np.sqrt(252)
        df['Vol_Scaler'] = (target_vol / df['Volatility']).clip(upper=1.5)
        
        vol_spike = df['Vol_Change'] > df['Vol_Change'].rolling(20).std() * 2
        df.loc[vol_spike, 'Vol_Scaler'] *= 0.5
        df.loc[df['Macro_Trend'] == 0, 'Vol_Scaler'] *= 0.5
        
        df['Signal'] = df['Signal'] * df['Vol_Scaler']
        self.data = df

## 4. NEW MODEL: STRATEGY V6

In [69]:
class StrategyV6_MetaLabeling(BaseStrategy):
    """
    V6.1 (Hybrid): The 'Regime-Adaptive' Institutional Model.
    
    Architecture:
    1. Primary Signal (Hybrid): 
       - TREND: Kalman Slope > 0 (Catch the run)
       - VALUE: RSI < 30 (Catch the dip)
       This ensures we have candidates in both trending and chopping markets.
       
    2. Meta-Labeling (Random Forest): 
       - Learns WHICH of the above signals works for the current asset/regime.
       
    3. Soft-Sizing: Scales leverage based on ML confidence.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.spy_data = None

    def fetch_data(self, warmup_years=2):
        super().fetch_data(warmup_years)
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        try:
            spy = yf.download("SPY", start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=False)
            if isinstance(spy.columns, pd.MultiIndex): spy.columns = spy.columns.get_level_values(0)
            spy['Macro_Trend'] = (spy['Adj Close'] > spy['Adj Close'].rolling(200).mean()).astype(int)
            self.spy_data = spy[['Macro_Trend']]
        except: pass

    def _apply_kalman_filter(self, prices):
        x = prices.values
        n = len(x)
        state = np.zeros(n)
        slope = np.zeros(n)
        state[0] = x[0]
        # Kalman Params
        P, Q, R = 1.0, 0.001, 0.1 
        
        for t in range(1, n):
            pred_state = state[t-1] + slope[t-1]
            pred_P = P + Q
            measurement = x[t]
            residual = measurement - pred_state
            
            K = pred_P / (pred_P + R)
            state[t] = pred_state + K * residual
            slope[t] = 0.9 * slope[t-1] + 0.1 * (state[t] - state[t-1])
            P = (1 - K) * pred_P
            
        return pd.Series(slope, index=prices.index)

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # --- 1. Features ---
        if self.spy_data is not None:
            df = df.join(self.spy_data, how='left').fillna(method='ffill')
        else: df['Macro_Trend'] = 1
            
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['Kalman_Slope'] = self._apply_kalman_filter(np.log(df['Adj Close']))
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df['Spread'] = df['Adj Close'] - df['Adj Close'].rolling(20).mean()
        df.dropna(inplace=True)
        
        # --- 2. Primary Signal (The Hybrid Generator) ---
        df['Primary_Signal'] = 0
        
        # A. MOMENTUM LEG (For NVDA/SPY)
        # Catch the trend when slope is positive
        trend_signal = (df['Kalman_Slope'] > 0)
        
        # B. MEAN REVERSION LEG (For JPM/Chop)
        # Catch the knife when oversold (Value)
        value_signal = (df['RSI'] < 30)
        
        # Combine: We are interested if EITHER is true
        df.loc[trend_signal | value_signal, 'Primary_Signal'] = 1
        
        # --- 3. Meta-Labeling (The Validator) ---
        # Label: Did buying here result in profit?
        labels = FeatureLab.triple_barrier_labels(df['Adj Close'], df['Volatility'], pt=1.0, sl=1.0, barrier_window=10)
        
        df['Meta_Prob'] = 0.5
        train_window = 252 * 2
        update_freq = 63 
        
        clf = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=42)
        feature_cols = ['Volatility', 'RSI', 'Spread', 'Kalman_Slope']
        
        indices = df.index
        if len(df) > train_window:
            for t in range(train_window, len(df), update_freq):
                train_start = indices[t - train_window]
                train_end = indices[t]
                test_end_idx = min(t + update_freq, len(df))
                test_end = indices[test_end_idx - 1]
                
                X_train = df.loc[train_start:train_end, feature_cols]
                y_train = labels.loc[train_start:train_end]
                
                # Training on all data allows the model to learn "High RSI = Good" for NVDA
                # and "Low RSI = Good" for JPM automatically based on recent history.
                clf.fit(X_train, y_train)
                
                X_test = df.loc[train_end:test_end, feature_cols]
                probs = clf.predict_proba(X_test)
                
                if probs.shape[1] == 2:
                    pos_probs = probs[:, 1]
                else:
                    pos_probs = probs[:, 0] if clf.classes_[0] == 1 else 0.0
                    
                df.loc[train_end:test_end, 'Meta_Prob'] = pos_probs
        
        # --- 4. Signal Construction ---
        df['Signal'] = 0.0
        
        # Confidence Floor: 
        # If the ML confirms the hybrid signal (Prob > 0.45), we execute.
        # This allows RSI Dips to pass IF the ML thinks they are profitable.
        active_trade = (df['Primary_Signal'] == 1) & (df['Meta_Prob'] > 0.45)
        df.loc[active_trade, 'Signal'] = 1
        
        # Sizing (Volatility + Confidence)
        target_vol = 0.15 / np.sqrt(252)
        vol_scaler = (target_vol / df['Volatility']).clip(upper=2.0)
        ml_scaler = (df['Meta_Prob'] / 0.5).clip(0.5, 2.0)
        
        # Macro Override
        # If Bear Market, we are defensive, BUT we allow Deep Value (RSI < 30) 
        # to have slightly more room if the ML loves it.
        macro_scaler = df['Macro_Trend'].map({1: 1.0, 0: 0.5})
        
        df['Signal'] = df['Signal'] * vol_scaler * ml_scaler * macro_scaler
        
        self.data = df

In [70]:
class StrategyV7_AdaptiveOptim(BaseStrategy):
    """
    V7 (WFO): Walk-Forward Optimized Strategy.
    
    Instead of static rules, this strategy runs a 'Tournament' every quarter.
    It tests 4 distinct parameter sets (Profiles) on the past 252 days:
    
    1. Trend_Aggro: Kalman Slope > 0 (No Macro Filter)
    2. Trend_Defense: Kalman Slope > 0 AND Macro_Bull (Like V3)
    3. Reversion_Deep: RSI < 30 (Buying Crashes)
    4. Reversion_Active: RSI < 45 (Buying Dips)
    
    It selects the Profile with the highest Sharpe Ratio in the lookback window
    and uses it for the next execution window.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.spy_data = None

    def fetch_data(self, warmup_years=2):
        super().fetch_data(warmup_years)
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        try:
            spy = yf.download("SPY", start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=False)
            if isinstance(spy.columns, pd.MultiIndex): spy.columns = spy.columns.get_level_values(0)
            spy['Macro_Trend'] = (spy['Adj Close'] > spy['Adj Close'].rolling(200).mean()).astype(int)
            self.spy_data = spy[['Macro_Trend']]
        except: pass

    def _apply_kalman_filter(self, prices):
        x = prices.values
        n = len(x)
        state = np.zeros(n)
        slope = np.zeros(n)
        state[0] = x[0]
        P, Q, R = 1.0, 0.01, 0.1 # Q=0.01 makes it slightly more responsive than V6
        
        for t in range(1, n):
            pred_state = state[t-1] + slope[t-1]
            pred_P = P + Q
            measurement = x[t]
            residual = measurement - pred_state
            
            K = pred_P / (pred_P + R)
            state[t] = pred_state + K * residual
            slope[t] = 0.9 * slope[t-1] + 0.1 * (state[t] - state[t-1])
            P = (1 - K) * pred_P
            
        return pd.Series(slope, index=prices.index)

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # --- 1. Global Feature Engineering ---
        if self.spy_data is not None:
            df = df.join(self.spy_data, how='left').fillna(method='ffill')
        else: df['Macro_Trend'] = 1
            
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['Kalman_Slope'] = self._apply_kalman_filter(np.log(df['Adj Close']))
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df.dropna(inplace=True)
        
        # --- 2. Pre-Calculate Strategy Candidates (Vectorized) ---
        # We calculate the raw signals for all profiles upfront
        
        # Profile 1: Aggressive Trend (Chase the move)
        sig_trend_aggro = (df['Kalman_Slope'] > 0).astype(int)
        
        # Profile 2: Defensive Trend (V3 Style - Only if Macro agrees)
        sig_trend_def = ((df['Kalman_Slope'] > 0) & (df['Macro_Trend'] == 1)).astype(int)
        
        # Profile 3: Deep Reversion (Catch Falling Knife)
        sig_rev_deep = (df['RSI'] < 30).astype(int)
        
        # Profile 4: Active Reversion (Buy Shallow Dips)
        sig_rev_active = (df['RSI'] < 45).astype(int)
        
        # Store in a dict for easy access
        candidates = {
            'Trend_Aggro': sig_trend_aggro,
            'Trend_Defense': sig_trend_def,
            'Rev_Deep': sig_rev_deep,
            'Rev_Active': sig_rev_active
        }
        
        # --- 3. Walk-Forward Optimization Loop ---
        df['Signal'] = 0.0
        df['Selected_Profile'] = 'None' # For debugging/analysis
        
        lookback = 252       # 1 Year Lookback for Optimization
        rebalance_freq = 63  # Quarterly Re-optimization
        
        indices = df.index
        daily_returns = df['Returns']
        
        if len(df) > lookback:
            for t in range(lookback, len(df), rebalance_freq):
                train_start = indices[t - lookback]
                train_end = indices[t]
                test_end_idx = min(t + rebalance_freq, len(df))
                test_end = indices[test_end_idx - 1]
                
                # The Tournament: Check Sharpe of each candidate in lookback period
                best_score = -999
                best_profile = 'Trend_Defense' # Default safety
                
                lb_returns = daily_returns.loc[train_start:train_end]
                
                for name, sig_series in candidates.items():
                    # Simulate Strategy Return in Lookback
                    # Lag signal by 1 to avoid lookahead in backtest
                    sigs = sig_series.loc[train_start:train_end].shift(1).fillna(0)
                    strat_ret = lb_returns * sigs
                    
                    # Calculate Metric (Sharpe)
                    mean_ret = strat_ret.mean()
                    std_ret = strat_ret.std()
                    
                    if std_ret > 1e-6:
                        score = mean_ret / std_ret # Simple Sharpe
                    else:
                        score = -999 # Flat line is bad
                        
                    if score > best_score:
                        best_score = score
                        best_profile = name
                
                # Apply Best Profile to Next Window (Test Set)
                # We use the signal series for the *future* window based on the *past* winner
                winner_signals = candidates[best_profile].loc[train_end:test_end]
                df.loc[train_end:test_end, 'Signal'] = winner_signals
                df.loc[train_end:test_end, 'Selected_Profile'] = best_profile

        # --- 4. Volatility Targeting (Risk Management) ---
        target_vol = 0.15 / np.sqrt(252)
        vol_scaler = (target_vol / df['Volatility']).clip(upper=1.5)
        
        df['Signal'] = df['Signal'] * vol_scaler
        self.data = df

In [71]:
class StrategyV8_GrandUnification(BaseStrategy):
    """
    V8: The Regime-Adaptive Ensemble.
    
    It combines the strengths of previous iterations:
    1. UNSUPERVISED STATE DETECTION (from V1): 
       Uses Gaussian Mixture Models (GMM) to classify the market into 3 regimes:
       - Low Vol / High Return -> "Stable Bull"
       - High Vol / Negative Return -> "Crisis/Bear"
       - Medium Vol / Flat Return -> "Chop"
       
    2. CONDITIONAL LOGIC (from V6):
       - If State == Stable Bull: Deploy AGGRESSIVE TREND (Kalman Slope).
       - If State == Chop: Deploy ACTIVE REVERSION (RSI < 45).
       - If State == Crisis: Go CASH (or Deep Value RSI < 25 only).
       
    3. GLOBAL SAFETY (from V3):
       - Overrides everything if SPY is below 200 SMA (Systemic Risk).
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.spy_data = None

    def fetch_data(self, warmup_years=2):
        super().fetch_data(warmup_years)
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        try:
            spy = yf.download("SPY", start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=False)
            if isinstance(spy.columns, pd.MultiIndex): spy.columns = spy.columns.get_level_values(0)
            spy['Macro_Trend'] = (spy['Adj Close'] > spy['Adj Close'].rolling(200).mean()).astype(int)
            self.spy_data = spy[['Macro_Trend']]
        except: pass

    def _apply_kalman_filter(self, prices):
        x = prices.values
        n = len(x)
        state = np.zeros(n)
        slope = np.zeros(n)
        state[0] = x[0]
        P, Q, R = 1.0, 0.001, 0.1
        for t in range(1, n):
            pred_state = state[t-1] + slope[t-1]
            pred_P = P + Q
            measurement = x[t]
            residual = measurement - pred_state
            K = pred_P / (pred_P + R)
            state[t] = pred_state + K * residual
            slope[t] = 0.9 * slope[t-1] + 0.1 * (state[t] - state[t-1])
            P = (1 - K) * pred_P
        return pd.Series(slope, index=prices.index)

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # --- 1. Data & Features ---
        if self.spy_data is not None:
            df = df.join(self.spy_data, how='left').fillna(method='ffill')
        else: df['Macro_Trend'] = 1
            
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['Kalman_Slope'] = self._apply_kalman_filter(np.log(df['Adj Close']))
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df['Returns_Smoothed'] = df['Returns'].rolling(5).mean()
        df['Vol_Smoothed'] = df['Volatility'].rolling(5).mean()
        df.dropna(inplace=True)
        
        # --- 2. GMM Regime Detection (The "Brain") ---
        # We cluster the market into 3 states based on Return & Risk
        X = df[['Returns_Smoothed', 'Vol_Smoothed']].values
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Train GMM
        gmm = GaussianMixture(n_components=3, random_state=42, n_init=10)
        df['Cluster'] = gmm.fit_predict(X_scaled)
        
        # --- 3. Dynamic Profile Mapping ---
        # We must figure out which cluster is which (Bull vs Bear vs Chop)
        # We assume:
        # - Highest Return = Bull
        # - Lowest Return = Bear
        # - Middle = Chop
        
        stats = df.groupby('Cluster')['Returns_Smoothed'].mean().sort_values()
        bear_cluster = stats.index[0]
        chop_cluster = stats.index[1]
        bull_cluster = stats.index[2]
        
        # Map clusters to readable names
        conditions = [
            (df['Cluster'] == bull_cluster),
            (df['Cluster'] == bear_cluster),
            (df['Cluster'] == chop_cluster)
        ]
        choices = ['BULL', 'BEAR', 'CHOP']
        df['Regime_Type'] = np.select(conditions, choices, default='CHOP')
        
        # --- 4. Regime-Conditional Signal Logic ---
        df['Signal'] = 0.0
        
        # A. BULL REGIME: Trend Following
        # Use Kalman Slope. If Slope > 0, we ride.
        # We ignore RSI overbought because strong trends stay overbought.
        bull_signal = (df['Regime_Type'] == 'BULL') & (df['Kalman_Slope'] > 0)
        df.loc[bull_signal, 'Signal'] = 1
        
        # B. CHOP REGIME: Mean Reversion
        # Market is going nowhere. Buy dips.
        # RSI < 45 is a good entry in chop.
        chop_signal = (df['Regime_Type'] == 'CHOP') & (df['RSI'] < 45)
        df.loc[chop_signal, 'Signal'] = 1
        
        # C. BEAR REGIME: Cash / Deep Value
        # Mostly Cash. Only buy EXTREME panic (RSI < 25).
        # This saved V1 on BABA.
        bear_signal = (df['Regime_Type'] == 'BEAR') & (df['RSI'] < 25)
        df.loc[bear_signal, 'Signal'] = 1
        
        # --- 5. Global Safety Filters ---
        
        # Filter 1: Macro Override (V3)
        # If the broad market is crashing (SPY < 200MA), reduce all long exposure by 50%
        # or cut entirely if it's a Bear stock in a Bear market.
        if 'Macro_Trend' in df.columns:
            # If Macro is bad, we kill the 'Chop' and 'Bull' signals for beta stocks
            # but we might keep 'Deep Value' signals.
            # For simplicity: Scale down everything.
            df.loc[df['Macro_Trend'] == 0, 'Signal'] *= 0.0 
            # STRICT RULE: No longs in Bear Market. This mimics V3's success.
        
        # Filter 2: Volatility Targeting
        target_vol = 0.15 / np.sqrt(252)
        vol_scaler = (target_vol / df['Volatility']).clip(upper=1.5)
        
        df['Signal'] = df['Signal'] * vol_scaler
        self.data = df

In [72]:
class StrategyV9_RegimeUnshackled(BaseStrategy):
    """
    V9: The 'Unshackled' Regime Model.
    
    Improvements over V8:
    1. TRUST THE BULL (Fixes JPM):
       - If GMM says 'Stable Bull', we go Long immediately.
       - Kalman Slope is demoted from a 'Gatekeeper' to a 'Sizing Booster'.
       
    2. THE ALPHA CLAUSE (Fixes NVDA):
       - If Stock is Bullish but SPY is Bearish (Macro Divergence), we do NOT exit.
       - We trade at 50% size. This captures relative strength leaders early.
       
    3. SURVIVAL MODE (Keeps BABA safe):
       - If GMM says 'Bear', we hard-exit to Cash (unless Deep Value RSI < 25).
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.spy_data = None

    def fetch_data(self, warmup_years=2):
        super().fetch_data(warmup_years)
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        try:
            spy = yf.download("SPY", start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=False)
            if isinstance(spy.columns, pd.MultiIndex): spy.columns = spy.columns.get_level_values(0)
            spy['Macro_Trend'] = (spy['Adj Close'] > spy['Adj Close'].rolling(200).mean()).astype(int)
            self.spy_data = spy[['Macro_Trend']]
        except: pass

    def _apply_kalman_filter(self, prices):
        x = prices.values
        n = len(x)
        state = np.zeros(n)
        slope = np.zeros(n)
        state[0] = x[0]
        P, Q, R = 1.0, 0.001, 0.1
        for t in range(1, n):
            pred_state = state[t-1] + slope[t-1]
            pred_P = P + Q
            measurement = x[t]
            residual = measurement - pred_state
            K = pred_P / (pred_P + R)
            state[t] = pred_state + K * residual
            slope[t] = 0.9 * slope[t-1] + 0.1 * (state[t] - state[t-1])
            P = (1 - K) * pred_P
        return pd.Series(slope, index=prices.index)

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # --- 1. Features ---
        if self.spy_data is not None:
            df = df.join(self.spy_data, how='left').fillna(method='ffill')
        else: df['Macro_Trend'] = 1
            
        df['Volatility'] = FeatureLab.yang_zhang_volatility(df)
        df['Kalman_Slope'] = self._apply_kalman_filter(np.log(df['Adj Close']))
        df['RSI'] = FeatureLab.compute_rsi(df['Adj Close'])
        df['Returns_Smoothed'] = df['Returns'].rolling(5).mean()
        df['Vol_Smoothed'] = df['Volatility'].rolling(5).mean()
        df.dropna(inplace=True)
        
        # --- 2. GMM Regime Detection ---
        X = df[['Returns_Smoothed', 'Vol_Smoothed']].values
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # We use warm_start=True logic simulation by consistent random_state
        gmm = GaussianMixture(n_components=3, random_state=42, n_init=10)
        df['Cluster'] = gmm.fit_predict(X_scaled)
        
        # Dynamic Mapping
        stats = df.groupby('Cluster')['Returns_Smoothed'].mean().sort_values()
        bear_cluster = stats.index[0]
        chop_cluster = stats.index[1]
        bull_cluster = stats.index[2]
        
        conditions = [
            (df['Cluster'] == bull_cluster),
            (df['Cluster'] == bear_cluster),
            (df['Cluster'] == chop_cluster)
        ]
        choices = ['BULL', 'BEAR', 'CHOP']
        df['Regime_Type'] = np.select(conditions, choices, default='CHOP')
        
        # --- 3. Unshackled Signal Logic ---
        df['Signal'] = 0.0
        
        # A. BULL REGIME: "Trust The Trend"
        # If GMM says Bull, we are Long. Period.
        # This captures JPM's "slow grind" that Kalman missed.
        bull_signal = (df['Regime_Type'] == 'BULL')
        df.loc[bull_signal, 'Signal'] = 1.0
        
        # Boost: If Kalman agrees (Strong Trend), we go 1.3x leverage
        strong_trend = bull_signal & (df['Kalman_Slope'] > 0)
        df.loc[strong_trend, 'Signal'] = 1.3
        
        # B. CHOP REGIME: "Active Trading"
        # Buy Dips.
        chop_buy = (df['Regime_Type'] == 'CHOP') & (df['RSI'] < 45)
        df.loc[chop_buy, 'Signal'] = 1.0
        
        # C. BEAR REGIME: "Survival"
        # Cash is King. Only buy extreme panic.
        panic_buy = (df['Regime_Type'] == 'BEAR') & (df['RSI'] < 25)
        df.loc[panic_buy, 'Signal'] = 1.0
        
        # --- 4. The Alpha Clause (Macro Handling) ---
        
        # Standard Volatility Sizing (Risk Parity)
        target_vol = 0.15 / np.sqrt(252)
        vol_scaler = (target_vol / df['Volatility']).clip(upper=1.5)
        
        # Macro Logic:
        # If SPY is Bearish (0), we don't kill the trade. We just HALVE it.
        # This allows NVDA to run while still being defensive.
        macro_scaler = df['Macro_Trend'].map({1: 1.0, 0: 0.5})
        
        df['Signal'] = df['Signal'] * vol_scaler * macro_scaler
        self.data = df

## QINGYANG LUCAS FANG

In [73]:
class U0_MeanReversion(BaseStrategy):
    """
    U0: Granular Mean Reversion (The "Wick Filter").
    Fixed for Yahoo Finance 730-day hourly limit.
    """
    def __init__(self, ticker, start_date, end_date, sensitivity=20, period=14, k=2.5):
        super().__init__(ticker, start_date, end_date)
        self.sensitivity = sensitivity
        self.period = period
        self.k = k

    def fetch_data(self, warmup_years=2):
        # NOTE: Yahoo Finance 1h data is STRICTLY limited to the last 730 days.
        # We must clamp the start_date to fit this constraint.
        
        # Calculate the requested start date with warmup
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        
        # Calculate the 730-day hard limit (with a small buffer)
        limit_dt = datetime.now() - timedelta(days=725)
        
        # Enforce limit
        if start_dt < limit_dt:
            print(f"Warning: {self.ticker} start date {start_dt.date()} exceeds 730-day hourly limit. Truncating to {limit_dt.date()}.")
            start_dt = limit_dt
        
        start_str = start_dt.strftime("%Y-%m-%d")
        
        try:
            # 1. Get Hourly Data (Feature Construction)
            df_h = yf.download(self.ticker, interval="1h", start=start_str, end=self.end_date, progress=False, auto_adjust=True)
            
            # Handle empty download immediately
            if df_h.empty:
                print(f"Error: No hourly data found for {self.ticker}")
                self.data = pd.DataFrame()
                return

            if isinstance(df_h.columns, pd.MultiIndex): df_h.columns = df_h.columns.get_level_values(0)
            df_h = df_h[['Close']].dropna()
            df_h.index = df_h.index.tz_localize(None)

            # 2. Resample to Daily
            daily_max = df_h['Close'].resample('1D').max()
            daily_min = df_h['Close'].resample('1D').min()
            daily_diff = daily_max - daily_min
            
            # 3. Get Standard Daily Data (Backtest alignment)
            df_d = yf.download(self.ticker, interval="1d", start=start_str, end=self.end_date, progress=False, auto_adjust=True)
            if df_d.empty:
                self.data = pd.DataFrame()
                return
                
            if isinstance(df_d.columns, pd.MultiIndex): df_d.columns = df_d.columns.get_level_values(0)
            df_d['Returns'] = df_d['Close'].pct_change()
            
            # 4. Merge
            df_d['Daily_Max_H'] = daily_max
            df_d['Daily_Min_H'] = daily_min
            df_d['Daily_Diff_H'] = daily_diff
            
            df_d.dropna(inplace=True)
            self.data = df_d
            
        except Exception as e:
            print(f"Error fetching data for U0 {self.ticker}: {e}")
            self.data = pd.DataFrame()

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # 1. Features
        df['ATR'] = df['Daily_Diff_H'].rolling(self.period).mean()
        df['Roll_Max'] = df['Daily_Max_H'].rolling(self.sensitivity).max()
        df['Roll_Min'] = df['Daily_Min_H'].rolling(self.sensitivity).min()
        
        # 2. SHIFT (No Look-Ahead)
        feat_cols = ['ATR', 'Roll_Max', 'Roll_Min', 'Daily_Max_H', 'Daily_Min_H']
        df[feat_cols] = df[feat_cols].shift(1)
        df.dropna(inplace=True)
        
        # 3. Bands
        df['Upper_Band'] = df['Roll_Max'] + self.k * df['ATR']
        df['Lower_Band'] = df['Roll_Min'] - self.k * df['ATR']
        
        # 4. Logic
        close = df['Close']
        long_entry = close < df['Lower_Band']
        short_entry = close > df['Upper_Band']
        long_exit = close > df['Daily_Max_H']
        short_exit = close < df['Daily_Min_H']
        
        # 5. State Machine
        pos = 0
        signals = []
        le_arr, se_arr = long_entry.values, short_entry.values
        lx_arr, sx_arr = long_exit.values, short_exit.values
        
        for i in range(len(df)):
            if pos == 0:
                if le_arr[i]: pos = 1
                elif se_arr[i]: pos = -1
            elif pos == 1:
                if lx_arr[i]: pos = 0
            elif pos == -1:
                if sx_arr[i]: pos = 0
            signals.append(pos)
            
        df['Signal'] = signals
        self.data = df

## 5. ROBUST BENCHMARK INFRASTRUCTURE

In [74]:
class RobustBenchmark:
    """
    Implements Walk-Forward Analysis and Deflated Sharpe Ratio logic.
    Benchmarks multiple strategies without look-ahead bias[cite: 275].
    """
    def __init__(self, tickers, start_date, end_date):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.results = []

    def run(self):
        print(f"{'STRATEGY':<10} | {'TICKER':<6} | {'ANN RET':<7} | {'SHARPE':<6} | {'MAX DD':<7} | {'NOTES'}")
        print("-" * 75)
        
        strategies = {
            "V1_Base": StrategyV1_Baseline,
            # "V2_GMM": StrategyV2_Advanced,
            "V3_Macro": StrategyV3_Macro,
            # "V4_Meta": StrategyV4_Meta,
            # "V5_Kalman": StrategyV5_KalmanState,
            # "V6_Inst": StrategyV6_MetaLabeling,
            "V7_Optim": StrategyV7_AdaptiveOptim,
            "V8_Final": StrategyV8_GrandUnification,
            "V9_Unshack": StrategyV9_RegimeUnshackled,
            "U0_MeanRev": U0_MeanReversion
        }

        for ticker in self.tickers:
            # Capture Buy & Hold first
            bh = StrategyV1_Baseline(ticker, self.start_date, self.end_date)
            bh.fetch_data()
            bh.data['Signal'] = 1 # Force Buy
            bh.run_backtest()
            self._print_row("Buy&Hold", ticker, bh.metrics)
            
            for name, StratClass in strategies.items():
                try:
                    strat = StratClass(ticker, self.start_date, self.end_date)
                    strat.fetch_data(warmup_years=2)
                    strat.generate_signals()
                    strat.run_backtest()
                    
                    self._print_row(name, ticker, strat.metrics)
                    
                    # Store for portfolio level (optional)
                    self.results.append({
                        'Ticker': ticker,
                        'Strategy': name,
                        'Returns': strat.results['Net_Returns']
                    })
                except Exception as e:
                    print(f"Failed {name} {ticker}: {e}")
            print("-" * 75)

    def _print_row(self, name, ticker, metrics):
        if not metrics: return
        ret = metrics['Total Return']
        # Annualize return approx
        ann_ret = (1 + ret) ** (252 / len(metrics.get('Returns', [1]*252))) - 1 if 'Returns' in metrics else ret
        print(f"{name:<10} | {ticker:<6} | {ret:.1%}   | {metrics['Sharpe Ratio']:.2f}   | {metrics['Max Drawdown']:.1%}   |")

## 6. EXECUTION

In [75]:
# tickers = [
#     "NVDA", # Tech Momentum
#     "JPM",  # Financial/Value
#     "TSLA", # High Volatility/Cult
#     "KO",   # Defensive/Staples
#     "MSTR", # Crypto/Hyper-Trend
#     "XLE",  # Energy/Cyclical
#     "BABA"  # Distressed/Bear
# ]

# bench = RobustBenchmark(
#     tickers=tickers, 
#     start_date="2022-01-01", 
#     end_date="2024-12-30"
# )
# bench.run()

## Ensemble

In [76]:
class Strategy_Ensemble(BaseStrategy):
    """
    The 'All-Weather' Ensemble.
    
    Combines V3 (Macro Trend) and V9 (Regime Unshackled) into a single
    portfolio-level signal.
    
    Logic:
    1. Runs V3 to capture high-beta trends (NVDA, Bitcoin).
    2. Runs V9 to capture regime-based alpha and protect downside (JPM, BABA).
    3. Blends signals using a 'Correlation-Adjusted' weighting or fixed 50/50.
    4. Applies a final Volatility Target to the combined equity curve to ensure
       the two strategies don't stack up to dangerous leverage.
    """
    def __init__(self, ticker, start_date, end_date, w_v3=0.5, w_v9=0.5):
        super().__init__(ticker, start_date, end_date)
        self.w_v3 = w_v3
        self.w_v9 = w_v9
        # Instantiate sub-strategies
        self.strat_v3 = StrategyV3_Macro(ticker, start_date, end_date)
        self.strat_v9 = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)

    def fetch_data(self, warmup_years=2):
        # Fetch once for efficiency (logic could be optimized to share DF, 
        # but separate fetch ensures cleaner encapsulation)
        self.strat_v3.fetch_data(warmup_years)
        self.strat_v9.fetch_data(warmup_years)
        
        # We share the index/data from one of them for the main wrapper
        if self.strat_v3.data is not None and not self.strat_v3.data.empty:
            self.data = self.strat_v3.data.copy()
        elif self.strat_v9.data is not None:
            self.data = self.strat_v9.data.copy()

    def generate_signals(self):
        if self.strat_v3.data is None or self.strat_v9.data is None: return
        
        # 1. Generate Sub-Signals
        self.strat_v3.generate_signals()
        self.strat_v9.generate_signals()
        
        # Align Indices (Inner Join to be safe)
        df = self.data.copy()
        s3 = self.strat_v3.data['Signal']
        s9 = self.strat_v9.data['Signal']
        
        # Merge signals into main DF
        df['Sig_V3'] = s3
        df['Sig_V9'] = s9
        df.dropna(inplace=True)
        
        # 2. The Allocation Logic
        # Default: Fixed Weight (Core-Satellite Approach)
        # V3 (Beta) + V9 (Alpha)
        
        # We blend the RAW signals.
        # Note: Signals are already Vol-Targeted to ~15% inside sub-classes.
        # Simple addition would double vol if correlation=1.
        raw_blend = (df['Sig_V3'] * self.w_v3) + (df['Sig_V9'] * self.w_v9)
        
        # 3. Ensemble Volatility Control
        # If V3 and V9 agree (both Long), we get high exposure.
        # If they disagree (V3 Long, V9 Cash), we get half exposure.
        # This naturally deleverages during uncertainty.
        
        df['Signal'] = raw_blend
        
        # Optional: Re-Target Volatility of the *Ensemble*
        # (Prevents leverage creep if strategies are highly correlated)
        # For now, we trust the weighted sum to act as a diversification benefit.
        
        self.data = df

In [77]:
class Strategy_Ensemble_Adaptive(BaseStrategy):
    """
    V10: The Adaptive Ensemble (Dynamic Weighting).
    
    Instead of fixed weights, this strategy re-allocates capital quarterly 
    based on the recent Risk-Adjusted Performance (Sharpe) of the sub-strategies.
    
    Logic:
    1. Lookback: 126 Days (6 Months).
    2. Rebalance: Every 63 Days (Quarterly).
    3. Weighting:
       - Calculate Sharpe Ratio for V3 and V9 in the lookback window.
       - If Sharpe > 0: Weight is proportional to Sharpe.
       - If Sharpe < 0: Weight is set to 0.
       - Normalize weights to sum to 1.0.
       
    This allows the portfolio to automatically 'Risk On' into V3 during strong bulls
    and 'Risk Off' into V9 during bears/chop.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        # Instantiate sub-strategies
        self.strat_v3 = StrategyV3_Macro(ticker, start_date, end_date)
        self.strat_v9 = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)

    def fetch_data(self, warmup_years=2):
        self.strat_v3.fetch_data(warmup_years)
        self.strat_v9.fetch_data(warmup_years)
        
        # Use one of the dataframes as the base
        if self.strat_v3.data is not None and not self.strat_v3.data.empty:
            self.data = self.strat_v3.data.copy()
        elif self.strat_v9.data is not None:
            self.data = self.strat_v9.data.copy()

    def generate_signals(self):
        if self.strat_v3.data is None or self.strat_v9.data is None: return
        
        # 1. Generate Sub-Signals
        self.strat_v3.generate_signals()
        self.strat_v9.generate_signals()
        
        # Merge Data
        df = self.data.copy()
        df['Sig_V3'] = self.strat_v3.data['Signal']
        df['Sig_V9'] = self.strat_v9.data['Signal']
        df.dropna(inplace=True)
        
        # 2. Simulate Sub-Strategy Returns (for metric calculation)
        # We need to know how they *would* have performed to weight them.
        # Lag signals by 1 to avoid lookahead bias when calculating returns.
        df['Ret_V3'] = df['Sig_V3'].shift(1) * df['Returns']
        df['Ret_V9'] = df['Sig_V9'].shift(1) * df['Returns']
        
        # 3. Walk-Forward Weight Optimization
        df['W_V3'] = 0.5 # Default start
        df['W_V9'] = 0.5
        
        lookback = 126      # 6 Months Lookback
        rebalance_freq = 21 # Monthly Rebalance (Faster adaptation)
        
        indices = df.index
        
        if len(df) > lookback:
            for t in range(lookback, len(df), rebalance_freq):
                train_start = indices[t - lookback]
                train_end = indices[t]
                test_end_idx = min(t + rebalance_freq, len(df))
                test_end = indices[test_end_idx - 1]
                
                # Calculate Sharpe in Lookback Window
                # Add small epsilon to std to avoid division by zero
                v3_mean = df.loc[train_start:train_end, 'Ret_V3'].mean()
                v3_std = df.loc[train_start:train_end, 'Ret_V3'].std() + 1e-9
                sharpe_v3 = (v3_mean / v3_std) * np.sqrt(252)
                
                v9_mean = df.loc[train_start:train_end, 'Ret_V9'].mean()
                v9_std = df.loc[train_start:train_end, 'Ret_V9'].std() + 1e-9
                sharpe_v9 = (v9_mean / v9_std) * np.sqrt(252)
                
                # Weighting Logic
                # 1. Filter: If Sharpe is negative, set score to 0
                score_v3 = max(0, sharpe_v3)
                score_v9 = max(0, sharpe_v9)
                
                # 2. Normalize
                total_score = score_v3 + score_v9
                
                if total_score > 0:
                    w_v3 = score_v3 / total_score
                    w_v9 = score_v9 / total_score
                else:
                    # Both are failing? Default to Defensive (V9) or Cash (0)
                    # Let's default to V9 (Safety) as the 'bunker'
                    w_v3 = 0.0
                    w_v9 = 1.0
                
                # Apply weights to NEXT window
                df.loc[train_end:test_end, 'W_V3'] = w_v3
                df.loc[train_end:test_end, 'W_V9'] = w_v9
                
        # 4. Final Signal Generation
        # Blend the signals using the dynamic weights
        df['Signal'] = (df['Sig_V3'] * df['W_V3']) + (df['Sig_V9'] * df['W_V9'])
        
        self.data = df

## HRP

In [78]:
class U0_MeanReversion(BaseStrategy):
    """
    U0: Hourly-Granularity Mean Reversion.
    Uses 'Wick-Free' Daily Highs/Lows derived from Hourly closes.
    """
    def __init__(self, ticker, start_date, end_date, sensitivity=20, period=14, k=2.5):
        super().__init__(ticker, start_date, end_date)
        self.sensitivity = sensitivity
        self.period = period
        self.k = k

    def fetch_data(self, warmup_years=2):
        # Clamp start date to Yahoo's 730-day hourly limit
        start_dt = datetime.strptime(self.start_date, "%Y-%m-%d") - timedelta(days=warmup_years*365)
        limit_dt = datetime.now() - timedelta(days=725)
        if start_dt < limit_dt: start_dt = limit_dt
        
        try:
            # 1. Hourly Data
            df_h = yf.download(self.ticker, interval="1h", start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=True)
            if df_h.empty: return
            if isinstance(df_h.columns, pd.MultiIndex): df_h.columns = df_h.columns.get_level_values(0)
            df_h = df_h[['Close']].dropna()
            df_h.index = df_h.index.tz_localize(None)

            # 2. Custom Daily Aggregation
            daily_max = df_h['Close'].resample('1D').max()
            daily_min = df_h['Close'].resample('1D').min()
            daily_diff = daily_max - daily_min
            
            # 3. Standard Daily Data
            df_d = yf.download(self.ticker, interval="1d", start=start_dt.strftime("%Y-%m-%d"), end=self.end_date, progress=False, auto_adjust=True)
            if isinstance(df_d.columns, pd.MultiIndex): df_d.columns = df_d.columns.get_level_values(0)
            
            df_d['Returns'] = df_d['Close'].pct_change()
            df_d['Daily_Max_H'] = daily_max
            df_d['Daily_Min_H'] = daily_min
            df_d['Daily_Diff_H'] = daily_diff
            df_d.dropna(inplace=True)
            self.data = df_d
        except: self.data = pd.DataFrame()

    def generate_signals(self):
        if self.data is None or self.data.empty: return
        df = self.data.copy()
        
        # Features
        df['ATR'] = df['Daily_Diff_H'].rolling(self.period).mean()
        df['Roll_Max'] = df['Daily_Max_H'].rolling(self.sensitivity).max()
        df['Roll_Min'] = df['Daily_Min_H'].rolling(self.sensitivity).min()
        
        # Shift (No Lookahead)
        cols = ['ATR', 'Roll_Max', 'Roll_Min', 'Daily_Max_H', 'Daily_Min_H']
        df[cols] = df[cols].shift(1)
        df.dropna(inplace=True)
        
        # Bands
        df['Upper'] = df['Roll_Max'] + self.k * df['ATR']
        df['Lower'] = df['Roll_Min'] - self.k * df['ATR']
        
        # State Machine Vectorization
        pos = 0
        signals = []
        close = df['Close'].values
        lower = df['Lower'].values
        upper = df['Upper'].values
        dmax = df['Daily_Max_H'].values
        dmin = df['Daily_Min_H'].values
        
        for i in range(len(df)):
            if pos == 0:
                if close[i] < lower[i]: pos = 1
                elif close[i] > upper[i]: pos = -1
            elif pos == 1:
                if close[i] > dmax[i]: pos = 0
            elif pos == -1:
                if close[i] < dmin[i]: pos = 0
            signals.append(pos)
            
        df['Signal'] = signals
        # Normalize signal to same scale as V3/V9 (approx 0 to 1 leverage)
        # U0 is aggressive, so we scale it down slightly
        df['Signal'] = df['Signal'] * 0.7 
        self.data = df

# ==========================================
# 3. HRP MATH ENGINE (The Allocator)
# ==========================================
class HRP_Allocator:
    @staticmethod
    def getIVP(cov):
        ivp = 1. / np.diag(cov)
        ivp /= ivp.sum()
        return ivp

    @staticmethod
    def getClusterVar(cov, cItems):
        cov_ = cov.loc[cItems, cItems] 
        w_ = HRP_Allocator.getIVP(cov_).reshape(-1, 1)
        cVar = np.dot(np.dot(w_.T, cov_), w_)[0, 0]
        return cVar

    @staticmethod
    def getQuasiDiag(link):
        link = link.astype(int)
        sortIx = pd.Series([link[-1, 0], link[-1, 1]])
        numItems = link[-1, 3] 
        while sortIx.max() >= numItems:
            sortIx.index = range(0, sortIx.shape[0] * 2, 2) 
            df0 = sortIx[sortIx >= numItems] 
            i = df0.index
            j = df0.values - numItems
            sortIx[i] = link[j, 0] 
            df0 = pd.Series(link[j, 1], index=i + 1)
            sortIx = sortIx.append(df0) 
            sortIx = sortIx.sort_index() 
            sortIx.index = range(sortIx.shape[0]) 
        return sortIx.tolist()

    @staticmethod
    def getRecBipart(cov, sortIx):
        w = pd.Series(1, index=sortIx)
        cItems = [sortIx] 
        while len(cItems) > 0:
            cItems = [i[j:k] for i in cItems for j, k in ((0, len(i) // 2), (len(i) // 2, len(i))) if len(i) > 1]
            for i in range(0, len(cItems), 2):
                cItems0 = cItems[i] 
                cItems1 = cItems[i + 1] 
                cVar0 = HRP_Allocator.getClusterVar(cov, cItems0)
                cVar1 = HRP_Allocator.getClusterVar(cov, cItems1)
                alpha = 1 - cVar0 / (cVar0 + cVar1)
                w[cItems0] *= alpha 
                w[cItems1] *= 1 - alpha 
        return w

    @staticmethod
    def optimize(returns_df):
        # Handle constant/zero returns to avoid NaN correlations
        if returns_df.std().min() < 1e-6:
            return pd.Series(1/len(returns_df.columns), index=returns_df.columns)
            
        corr = returns_df.corr().fillna(0)
        cov = returns_df.cov().fillna(0)
        dist = ((1 - corr) / 2.) ** .5
        link = sch.linkage(dist, 'single')
        sortIx = HRP_Allocator.getQuasiDiag(link)
        sortIx = corr.index[sortIx].tolist()
        return HRP_Allocator.getRecBipart(cov, sortIx)

class Strategy_Ensemble_HRP(BaseStrategy):
    """
    Phase 1 Upgrade: The HRP Ensemble.
    Allocates capital dynamically between V3 (Trend), V9 (Regime), and U0 (Mean Rev)
    based on their CORRELATION structure, not just returns.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.v3 = StrategyV3_Macro(ticker, start_date, end_date)
        self.v9 = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        self.u0 = U0_MeanReversion(ticker, start_date, end_date)
    
    def fetch_data(self, warmup_years=2):
        self.u0.fetch_data(warmup_years) # U0 has strictest data reqs
        if self.u0.data is None or self.u0.data.empty: return
        
        # Sync V3/V9 to U0's available timeframe
        valid_start = self.u0.data.index[0].strftime("%Y-%m-%d")
        self.v3.start_date = valid_start
        self.v9.start_date = valid_start
        
        self.v3.fetch_data(warmup_years)
        self.v9.fetch_data(warmup_years)
        self.data = self.u0.data.copy() # Use U0 as master index

    def generate_signals(self):
        if self.u0.data is None: return
        
        # Run Sub-Strategies
        self.v3.generate_signals()
        self.v9.generate_signals()
        self.u0.generate_signals()
        
        # Merge Signals
        df = self.data.copy()
        # Join by index to handle missing rows
        df = df.join(self.v3.data[['Signal']].rename(columns={'Signal':'S_V3'}), how='inner')
        df = df.join(self.v9.data[['Signal']].rename(columns={'Signal':'S_V9'}), how='inner')
        df = df.join(self.u0.data[['Signal']].rename(columns={'Signal':'S_U0'}), how='inner')
        
        # Simulate Returns for HRP
        df['R_V3'] = df['S_V3'].shift(1) * df['Returns']
        df['R_V9'] = df['S_V9'].shift(1) * df['Returns']
        df['R_U0'] = df['S_U0'].shift(1) * df['Returns']
        
        # HRP Walk-Forward Loop
        lookback = 63  # Short lookback for correlation (3 months)
        rebal = 21     # Monthly rebalance
        
        df['W_V3'], df['W_V9'], df['W_U0'] = 0.33, 0.33, 0.33
        
        indices = df.index
        for t in range(lookback, len(df), rebal):
            start = indices[t-lookback]
            end = indices[t]
            test_end = indices[min(t+rebal, len(df)-1)]
            
            # Extract Returns History for Optimization
            ret_hist = df.loc[start:end, ['R_V3', 'R_V9', 'R_U0']]
            
            # Run HRP
            try:
                weights = HRP_Allocator.optimize(ret_hist)
                df.loc[end:test_end, 'W_V3'] = weights['R_V3']
                df.loc[end:test_end, 'W_V9'] = weights['R_V9']
                df.loc[end:test_end, 'W_U0'] = weights['R_U0']
            except: pass
            
        # Final Ensemble Signal
        df['Signal'] = (df['S_V3']*df['W_V3']) + (df['S_V9']*df['W_V9']) + (df['S_U0']*df['W_U0'])
        self.data = df

## HRP Momentum

In [79]:
class StrategyV11_HRP_Momentum(BaseStrategy):
    """
    V11: HRP + Momentum (The 'Smart' Allocator).
    
    Fixes the 'Conservative Bias' of standard HRP.
    
    Logic:
    1. Risk Engine (HRP): Calculates base weights to minimize portfolio variance 
       and handle correlation clustering (separating Trend vs. Mean Rev).
    2. Return Engine (Momentum): Calculates the rolling Sharpe Ratio of each strategy.
    3. Fusion: 
       Final_Weight = HRP_Weight * (1 + Sharpe_Score)
       
    This ensures we diversify risk BUT aggressively overweight the strategies 
    that are actually making money right now.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.v3 = StrategyV3_Macro(ticker, start_date, end_date)
        self.v9 = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        self.u0 = U0_MeanReversion(ticker, start_date, end_date) # Our uncorrelated hedge

    def fetch_data(self, warmup_years=2):
        # Fetch U0 first (strict data limits)
        self.u0.fetch_data(warmup_years)
        if self.u0.data is None or self.u0.data.empty: return
        
        valid_start = self.u0.data.index[0].strftime("%Y-%m-%d")
        self.v3.start_date = valid_start
        self.v9.start_date = valid_start
        
        self.v3.fetch_data(warmup_years)
        self.v9.fetch_data(warmup_years)
        self.data = self.u0.data.copy()

    def generate_signals(self):
        if self.u0.data is None: return
        
        # 1. Run Sub-Strategies
        self.v3.generate_signals()
        self.v9.generate_signals()
        self.u0.generate_signals()
        
        # 2. Merge Signals & Sync Index
        df = self.data.copy()
        df = df.join(self.v3.data[['Signal']].rename(columns={'Signal':'S_V3'}), how='inner')
        df = df.join(self.v9.data[['Signal']].rename(columns={'Signal':'S_V9'}), how='inner')
        df = df.join(self.u0.data[['Signal']].rename(columns={'Signal':'S_U0'}), how='inner')
        
        # 3. Simulate Returns (Needed for Correlation & Sharpe)
        df['R_V3'] = df['S_V3'].shift(1) * df['Returns']
        df['R_V9'] = df['S_V9'].shift(1) * df['Returns']
        df['R_U0'] = df['S_U0'].shift(1) * df['Returns']
        
        # 4. HRP-Momentum Optimization Loop
        lookback = 63  # 3 Months for correlation structure
        rebal = 21     # Monthly rebalance
        
        # Initialize with equal weights (fallback)
        df['W_V3'], df['W_V9'], df['W_U0'] = 0.33, 0.33, 0.33
        
        indices = df.index
        # Start loop only after lookback is satisfied
        if len(df) > lookback:
            for t in range(lookback, len(df), rebal):
                start = indices[t-lookback]
                end = indices[t]
                test_end = indices[min(t+rebal, len(df)-1)]
                
                # A. Get Returns History
                hist = df.loc[start:end, ['R_V3', 'R_V9', 'R_U0']]
                
                # B. Calculate HRP Base Weights (Risk Parity)
                # This minimizes the 'Cluster Variance'
                try:
                    hrp_w = HRP_Allocator.optimize(hist)
                except:
                    hrp_w = pd.Series([0.33, 0.33, 0.33], index=['R_V3','R_V9','R_U0'])
                
                # C. Calculate Momentum Boost (Sharpe Ratio)
                means = hist.mean()
                stds = hist.std() + 1e-9
                sharpes = (means / stds) * np.sqrt(252)
                
                # Clip negative Sharpes to 0 (don't fund losers)
                mom_score = sharpes.clip(lower=0) 
                
                # D. Fusion Logic
                # Combine Risk Weight * (1 + Momentum Score)
                # This boosts high-sharpe assets while respecting correlation clusters
                raw_w = hrp_w * (1 + mom_score)
                
                # Normalize to sum to 1.0
                final_w = raw_w / raw_w.sum()
                
                # Fill NaNs (if sum was 0) with Safety (V9)
                if final_w.isnull().any():
                    final_w = pd.Series({'R_V3':0.0, 'R_V9':1.0, 'R_U0':0.0})
                    
                # Apply weights
                df.loc[end:test_end, 'W_V3'] = final_w['R_V3']
                df.loc[end:test_end, 'W_V9'] = final_w['R_V9']
                df.loc[end:test_end, 'W_U0'] = final_w['R_U0']

        # 5. Final Signal
        df['Signal'] = (df['S_V3']*df['W_V3']) + (df['S_V9']*df['W_V9']) + (df['S_U0']*df['W_U0'])
        self.data = df

In [80]:
class StrategyV12_Macro_Switch(BaseStrategy):
    """
    V12: The Macro-Guided Ensemble.
    
    Replaces lookback windows with Real-Time Economic Data.
    
    DATA SOURCES (Yahoo Finance):
    1. ^VIX: CBOE Volatility Index.
    2. ^TNX: 10-Year Treasury Yield.
    
    LOGIC:
    1. Calculate 'Macro Stress Score' (0.0 to 1.0).
       - VIX Component: Normalized against recent history. High VIX = High Stress.
       - Yield Component: Rate of Change (ROC) of TNX. Spiking rates = High Stress.
    
    2. Dynamic Weighting:
       - Weight_V3 (Trend) = 1.0 - Stress_Score
       - Weight_V9 (Safety) = Stress_Score
       
    HYPOTHESIS:
    VIX and Rates often spike BEFORE the price crash is fully realized. 
    This allows the model to switch to safety faster than a Moving Average.
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.v3 = StrategyV3_Macro(ticker, start_date, end_date)
        self.v9 = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        # We store macro data separately
        self.macro_data = None

    def fetch_data(self, warmup_years=2):
        # 1. Fetch Ticker Data
        self.v3.fetch_data(warmup_years)
        self.v9.fetch_data(warmup_years)
        
        if self.v3.data is None or self.v3.data.empty: return
        self.data = self.v3.data.copy()
        
        # 2. Fetch Macro Data (VIX and TNX)
        start_dt = (self.data.index[0] - timedelta(days=365)).strftime("%Y-%m-%d")
        end_dt = self.end_date
        
        try:
            vix = yf.download("^VIX", start=start_dt, end=end_dt, progress=False, auto_adjust=True)
            tnx = yf.download("^TNX", start=start_dt, end=end_dt, progress=False, auto_adjust=True)
            
            # Cleaning
            if isinstance(vix.columns, pd.MultiIndex): vix.columns = vix.columns.get_level_values(0)
            if isinstance(tnx.columns, pd.MultiIndex): tnx.columns = tnx.columns.get_level_values(0)
            
            macro_df = pd.DataFrame(index=self.data.index)
            # Align macro data to the ticker's trading days (ffill for holidays)
            macro_df['VIX'] = vix['Close'].reindex(self.data.index, method='ffill')
            macro_df['TNX'] = tnx['Close'].reindex(self.data.index, method='ffill')
            
            self.macro_data = macro_df
            
        except Exception as e:
            print(f"Macro Data Fetch Error: {e}")
            # Fallback: Zero stress
            self.macro_data = pd.DataFrame({'VIX': 20, 'TNX': 4}, index=self.data.index)

    def generate_signals(self):
        if self.data is None or self.macro_data is None: return
        
        # 1. Run Sub-Strategies
        self.v3.generate_signals()
        self.v9.generate_signals()
        
        # 2. Sync Data
        df = self.data.copy()
        df = df.join(self.v3.data[['Signal']].rename(columns={'Signal':'S_V3'}), how='left')
        df = df.join(self.v9.data[['Signal']].rename(columns={'Signal':'S_V9'}), how='left')
        
        # 3. Calculate Macro Stress Score
        macro = self.macro_data.copy()
        
        # A. VIX Stress (Fear)
        # Normalize VIX: If VIX > 30, Stress = 1.0. If VIX < 15, Stress = 0.0.
        # Uses a rolling Z-score or simple clamp? Simple clamp is more robust to regime shifts.
        macro['VIX_Stress'] = ((macro['VIX'] - 15) / (30 - 15)).clip(0, 1)
        
        # B. Yield Stress (Rate Shock)
        # We care about SPEED of rate rise, not just level.
        # Calculate 20-day Rate of Change of TNX
        macro['TNX_ROC'] = macro['TNX'].pct_change(20)
        # If Yields rise > 10% in a month, that's a shock.
        macro['TNX_Stress'] = (macro['TNX_ROC'] / 0.10).clip(0, 1)
        
        # Combined Stress (Max of either Fear or Rate Shock)
        # We use Max because either one can crash the market independently.
        macro['Total_Stress'] = macro[['VIX_Stress', 'TNX_Stress']].max(axis=1)
        
        # 4. Allocate Weights
        # Smooth the stress signal to avoid daily jitter (3-day avg)
        stress_signal = macro['Total_Stress'].rolling(3).mean().fillna(0)
        
        df['W_V9'] = stress_signal        # High Stress -> More Safety
        df['W_V3'] = 1.0 - stress_signal  # Low Stress -> More Trend
        
        # 5. Final Signal
        df['Signal'] = (df['S_V3'] * df['W_V3']) + (df['S_V9'] * df['W_V9'])
        
        self.data = df

In [81]:
class StrategyV13_Correlation_Switch(BaseStrategy):
    """
    V13: The Correlation-Aware 'Router'.
    
    Dynamically switches between V12 (Macro) and V9 (Internal Regime) based on 
    the asset's sensitivity to Market Stress (VIX).
    
    LOGIC:
    1. Calculate Rolling Correlation (6-Month) between Asset Returns and VIX Changes.
    2. The 'Fragility' Test:
       - If Corr(Asset, VIX) < -0.2: Asset crashes when Fear spikes.
         -> ROUTE TO V12 (Macro-Guided).
       - If Corr(Asset, VIX) >= -0.2: Asset ignores/benefits from Fear.
         -> ROUTE TO V9 (Regime Unshackled).
         
    This ensures we use the Macro Filter for NVDA (where it works) 
    but turn it off for XLE/BABA (where it fails).
    """
    def __init__(self, ticker, start_date, end_date):
        super().__init__(ticker, start_date, end_date)
        self.v12 = StrategyV12_Macro_Switch(ticker, start_date, end_date)
        self.v9 = StrategyV9_RegimeUnshackled(ticker, start_date, end_date)
        self.macro_data = None

    def fetch_data(self, warmup_years=2):
        # 1. Fetch Sub-Strategies (V12 fetches Macro Data internally)
        self.v12.fetch_data(warmup_years)
        self.v9.fetch_data(warmup_years)
        
        if self.v12.data is None or self.v12.data.empty: return
        self.data = self.v12.data.copy()
        
        # 2. Extract Macro Data (VIX) from V12 instance
        if self.v12.macro_data is not None:
            # Reindex to match main data exactly
            self.macro_data = self.v12.macro_data.reindex(self.data.index).fillna(method='ffill')
        else:
            # Fallback
            self.macro_data = pd.DataFrame({'VIX': 20}, index=self.data.index)

    def generate_signals(self):
        if self.data is None or self.macro_data is None: return
        
        # 1. Run Sub-Strategies
        self.v12.generate_signals()
        self.v9.generate_signals()
        
        # 2. Merge Signals
        df = self.data.copy()
        df = df.join(self.v12.data[['Signal']].rename(columns={'Signal':'S_V12'}), how='left')
        df = df.join(self.v9.data[['Signal']].rename(columns={'Signal':'S_V9'}), how='left')
        
        # 3. Calculate Correlation Logic
        # We check correlation between Asset Daily Returns and VIX Daily Changes
        df['Ret'] = df['Returns']
        df['VIX_Chg'] = self.macro_data['VIX'].pct_change()
        
        # Rolling Correlation (126 Days = ~6 Months)
        # We want to know: "In the last 6 months, did this stock act like a Beta stock?"
        df['VIX_Corr'] = df['Ret'].rolling(126).corr(df['VIX_Chg'])
        
        # 4. The Router Switch
        # Threshold: -0.2. 
        # If corr is more negative than -0.2 (e.g., -0.5), it means strong inverse relationship.
        # High Fear = Low Price. -> Use V12.
        # If corr is > -0.2 (e.g., 0.1), relation is weak/positive. -> Use V9.
        
        # Lag the switch by 1 day to avoid lookahead
        switch_signal = df['VIX_Corr'].shift(1).fillna(-1.0) # Default to V12 (Safety)
        
        df['Weight_V12'] = np.where(switch_signal < -0.2, 1.0, 0.0)
        df['Weight_V9']  = 1.0 - df['Weight_V12']
        
        # 5. Final Signal
        df['Signal'] = (df['S_V12'] * df['Weight_V12']) + (df['S_V9'] * df['Weight_V9'])
        
        self.data = df

## Execution

In [83]:
if __name__ == "__main__":
    print(f"{'STRATEGY':<12} | {'TICKER':<6} | {'ANN RET':<7} | {'SHARPE':<6} | {'MAX DD':<7} | {'NOTES'}")
    print("-" * 79)
    
    # Helper wrappers for the dictionary
    class Strategy_Ensemble_5050(Strategy_Ensemble):
        def __init__(self, ticker, start, end): super().__init__(ticker, start, end, 0.5, 0.5)

    class Strategy_Ensemble_Growth(Strategy_Ensemble):
        def __init__(self, ticker, start, end): super().__init__(ticker, start, end, 0.7, 0.3)

    strategies = {
        "V3_Macro": StrategyV3_Macro,
        "V9_Unshack": StrategyV9_RegimeUnshackled,
        # "Ens_Bal": Strategy_Ensemble_5050,      # Static 50/50
        # "Ens_Grow": Strategy_Ensemble_Growth,   # Static 70/30
        "Ens_Adapt": Strategy_Ensemble_Adaptive, # Dynamic V10
        # "HRP_Base": Strategy_Ensemble_HRP,
        "V11_HRP_Mom": StrategyV11_HRP_Momentum,
        "V12_Macro": StrategyV12_Macro_Switch,
        "V13_Switch": StrategyV13_Correlation_Switch,
    }

    # Same Stress Test Basket
    tickers = ["NVDA", "JPM", "TSLA", "BABA", "XLE"]

    bench = RobustBenchmark(
        tickers=tickers, 
        start_date="2022-01-01", 
        end_date="2024-12-30"
    )
    
    # Manual run loop to handle the specific classes
    for ticker in tickers:
        # Buy & Hold
        bh = StrategyV1_Baseline(ticker, bench.start_date, bench.end_date)
        bh.fetch_data()
        bh.data['Signal'] = 1
        bh.run_backtest()
        bench._print_row("Buy&Hold", ticker, bh.metrics)
        
        for name, StratClass in strategies.items():
            try:
                strat = StratClass(ticker, bench.start_date, bench.end_date)
                strat.fetch_data(warmup_years=2)
                strat.generate_signals()
                strat.run_backtest()
                bench._print_row(name, ticker, strat.metrics)
            except Exception as e:
                print(f"Err {name} {ticker}: {e}")
        print("-" * 79)

STRATEGY     | TICKER | ANN RET | SHARPE | MAX DD  | NOTES
-------------------------------------------------------------------------------
Buy&Hold   | NVDA   | 355.4%   | 1.19   | -62.7%   |
V3_Macro   | NVDA   | 173.4%   | 1.51   | -23.1%   |
V9_Unshack | NVDA   | 30.4%   | 0.98   | -9.0%   |
Ens_Adapt  | NVDA   | 85.2%   | 1.23   | -22.0%   |
V11_HRP_Mom | NVDA   | 22.9%   | 1.34   | -10.4%   |
V12_Macro  | NVDA   | 125.0%   | 1.76   | -11.2%   |
V13_Switch | NVDA   | 125.0%   | 1.76   | -11.2%   |
-------------------------------------------------------------------------------
Buy&Hold   | JPM    | 62.1%   | 0.77   | -37.9%   |
V3_Macro   | JPM    | 71.9%   | 0.97   | -28.5%   |
V9_Unshack | JPM    | 84.4%   | 0.95   | -25.1%   |
Ens_Adapt  | JPM    | 75.6%   | 0.93   | -26.9%   |
V11_HRP_Mom | JPM    | 19.8%   | 1.42   | -8.0%   |
V12_Macro  | JPM    | 74.6%   | 0.92   | -24.8%   |
V13_Switch | JPM    | 74.6%   | 0.92   | -24.8%   |
-------------------------------------------------